In [1]:
# Import Dependencies
import pandas as pd

In [5]:
#create data frames from csv
schools_df = pd.read_csv('Resources/schools_complete.csv')
students_df = pd.read_csv('Resources/students_complete.csv')

In [7]:
# Merge data frames  
schools_full_df = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])
schools_full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [45]:
## District Summary
#Calculate the total number of schools
num_schools = len(schools_df)

# Calculate the total number of students
num_students = len(students_df)

# Calculate the total budget
total_budget = schools_df['budget'].sum()

# Calculate the average math score 
math_score_ave = students_df['math_score'].mean()

# Calculate the average reading score
read_score_ave = students_df['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
math_pass_df = students_df.loc[students_df['math_score'] >= 70, :]
per_pass_math = len(math_score)
per_pass_math = per_pass_math / num_students * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
per_pass_read = len(students_df.loc[students_df['reading_score'] >= 70, :])
per_pass_read = per_pass_read / num_students * 100

# Calculate the percentage of students who passed math **and** reading (% Overall Passing)
per_pass = len(math_pass_df.loc[math_pass_df['reading_score'] >= 70, :])
per_pass = per_pass / num_students * 100

# Create a dataframe to hold the above results
district_df = pd.DataFrame({'Total schools' : [num_schools],
             'Total students' : num_students,
             'Total budget' : total_budget,
             'Average math score' : math_score_ave,
             'Average reading score' : read_score_ave,
             '% Passing math' : per_pass_math,
             '% Passing reading' : per_pass_read,
             '% Passing both' : per_pass})

#Proper formatting
district_df['Total schools'] = district_df['Total schools'].map('{:,}'.format)
district_df['Total students'] = district_df['Total students'].map('{:,}'.format)
district_df['Total budget'] = district_df['Total budget'].map('${:,.2f}'.format)
district_df['Average math score'] = district_df['Average math score'].map('{:,.2f}'.format)
district_df['Average reading score'] = district_df['Average reading score'].map('{:,.2f}'.format)
district_df['% Passing math'] = district_df['% Passing math'].map('{:,.2f}%'.format)
district_df['% Passing reading'] = district_df['% Passing reading'].map('{:,.2f}%'.format)
district_df['% Passing both'] = district_df['% Passing both'].map('{:,.2f}%'.format)

district_df

,Total schools,Total students,Total budget,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%
